In [1]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, A4, landscape
from reportlab.platypus import Table, Image, Paragraph, Spacer, PageTemplate, Frame, TableStyle, SimpleDocTemplate
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.pdfgen.canvas import Canvas
from datetime import datetime
import socket
import geocoder
import reverse_geocoder as rg
import hashlib
import io

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def capturando_localizacao():
    g = geocoder.ip('me')
    results = rg.search((g.latlng[0], g.latlng[1]))
    localizacao = results[0]  # Assume que o primeiro resultado é o mais relevante
    estado = localizacao['admin1']
    pais = localizacao['cc']
    return estado + ' - ' + pais

# gerar hash do documento em memória
def generate_pdf_hash(file_path):
    hasher = hashlib.sha256()
    pdf_buffer.seek(0)
    buf = pdf_buffer.read()
    hasher.update(buf)
    return hasher.hexdigest()

# Função para desenhar o rodapé
def adicionando_rodape(canvas: Canvas, doc):
    city = socket.gethostname()  # Obtendo o nome da máquina como cidade
    now = datetime.now()
    date_str = now.strftime("%d/%m/%Y %H:%M")
    
    empresa = "Guirra Datatec"
    empresa_slogan = "Transformando dados em decisões"
    
    footer_text = f"Localização: {capturando_localizacao()}, Máquina: {city}, Data e Hora: {date_str}"
    footer_text_hash = f"{pdf_hash}"
    footer_text_emp = f"{empresa} - {empresa_slogan}"

    canvas.saveState()
    canvas.setFont('Helvetica', 8.5)
    tam_pag = landscape(A4)[0]  # Usando a largura da página em modo paisagem
    
    tam_text_loc = canvas.stringWidth(footer_text, 'Helvetica', 8.5)
    posicao_centro_loc = (tam_pag - tam_text_loc) / 2
    canvas.drawString(posicao_centro_loc, 40, footer_text)
    
    tam_text_hash = canvas.stringWidth(footer_text_hash, 'Helvetica', 8.5)
    posicao_centro_hash = (tam_pag - tam_text_hash) / 2
    canvas.drawString(posicao_centro_hash, 25, footer_text_hash)
    
    tam_text_emp = canvas.stringWidth(footer_text_emp, 'Helvetica', 8.5)
    posicao_centro = (tam_pag - tam_text_emp) / 2
    canvas.drawString(posicao_centro, 10, footer_text_emp)
    canvas.restoreState()
    
# Função para retornar a imagem baseada no valor (0 ou 1)
def insere_resultado(value):
    img_sucess = "/home/fernando/Área de Trabalho/WEBSERVICE - SEFAZ/RELATORIO/img/aceitar.png"
    img_fail = "/home/fernando/Área de Trabalho/WEBSERVICE - SEFAZ/RELATORIO/img/cancelar.png"
    img_pause = "/home/fernando/Área de Trabalho/WEBSERVICE - SEFAZ/RELATORIO/img/relogio.png"
    
    if value == '1':
        return Image(img_sucess, width=25, height=25)
    elif value == '0':
        return Image(img_fail, width=25, height=25)
    return Image(img_pause, width=25, height=25)

# função para tratamento da lista de retornos iniciais que são passados, referentes ao envios do documento
def trata_lista_retornos(lista):
    linha = 1
    for lst in lista:
        lst.insert(0, str(linha))
        lst[2] = insere_resultado(lst[-1])
        linha += 1
        
    return lista

In [72]:
def gera_documento_final():
    # Definindo as margens (esquerda, direita, superior, inferior)
    left_margin = 20
    right_margin = 30
    top_margin = 30
    bottom_margin = 20

    # Criando o documento com margens definidas e orientação horizontal
    pdf = SimpleDocTemplate(
        "Relatorio_Consulta.pdf",
        pagesize=A4,
        leftMargin=left_margin,
        rightMargin=right_margin,
        topMargin=top_margin,
        bottomMargin=bottom_margin
    )

    # Caminhos das imagens
    img_pathd = "/home/fernando/Downloads/GD_1-removebg-preview.png"
    img_pathe = "/home/fernando/Downloads/GD_1-removebg-preview.png"

    # Criando as imagens
    img_esq = Image(img_pathe, width=85, height=85)
    img_dir = Image(img_pathd, width=85, height=85)
    
    # definindo estilo html - paragraph
    styles = getSampleStyleSheet()
    # Estilos para diferentes tags HTML
    h1_style = ParagraphStyle(
        name='H1Style',
        parent=styles['Heading1'],
        fontSize=14,
        leading=20,
        alignment=1,  # Center alignment
        textColor=colors.whitesmoke
    )

    h3_style = ParagraphStyle(
        name='H3Style',
        parent=styles['Heading3'],
        fontSize=12,
        leading=5,
        alignment=1,  # Center alignment
        textColor=colors.whitesmoke
    )

    h4_style = ParagraphStyle(
        name='H4Style',
        parent=styles['Heading4'],
        fontSize=10,
        leading=14,
        alignment=1,  # Center alignment
        textColor=colors.whitesmoke
    )
    
    titulos_doc = ParagraphStyle(
        name='H4Style',
        parent=styles['Bullet'],
        fontSize=10,
        leading=0,
        alignment=0,
        textColor=colors.black
    )
    
    # Criando parágrafos estilizados com os estilos definidos
    paragraph1 = Paragraph("Relatório Geral da Consulta", h1_style)
    paragraph2 = Paragraph("Ambiente", h3_style)
    paragraph3 = Paragraph("Tipo de Serviço", h4_style)

    # Agrupando os parágrafos em uma lista para inclusão na célula da tabela
    paragraphs = [Spacer(1, 1), paragraph1, Spacer(1, 1), paragraph2, Spacer(1, 3), paragraph3]

    # Dados da tabela
    data = [
        [img_esq, paragraphs, img_dir]
    ]

    # Definindo as larguras das colunas
    col_widths = [110, 330, 110]  # Larguras em pontos (1 ponto = 1/72 polegada)

    # Criando a tabela com larguras definidas
    table = Table(data, colWidths=col_widths)
    
    # Adicionando estilo à tabela
    style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#111111")),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor("#FFFFFF")),
        ('ALIGN', (0, 0), (2, 0), 'CENTER'),
        ('VALIGN', (0, 0), (2, 0), 'MIDDLE'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.HexColor("#111111"))
    ])
    table.setStyle(style)
    
    elements = [
        table
    ]
    
    # Adicionar um título
    elements.append(Spacer(1, 15))
    title_dados_paciente = Paragraph("Dados do Paciente", titulos_doc)
    elements.append(title_dados_paciente)

    # Adicionando o template com o rodapé
    frame = Frame(pdf.leftMargin, pdf.bottomMargin + 30, pdf.width, pdf.height - 10, id='normal')
    template = PageTemplate(id='test', frames=[frame], onPage=adicionando_rodape)
    pdf.addPageTemplates([template])
    
    pdf.build(elements)

In [73]:
gera_documento_final()